<a href="https://colab.research.google.com/github/p-perrone/UiO_AdvancedRemoteSensing/blob/main/Project1_sketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import packages
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
import xarray as xr

In [ ]:
# Run authentication
ee.Authenticate()
ee.Initialize(project='dulcet-iterator-470310-n0') # PUT YOUR API KEY (Project ID) HERE!

# Part 1: loading data

In [ ]:
# this is for getting rid of edges (usually artifacted)
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)

# testing geometry
sea_ice_geom = ee.Geometry.Polygon(
    [[[2501471, 14525773],
      [2572610, 14525773],
      [2572610, 14566293],
      [2501471, 14566293],
      [2501471, 14525773]]],
    proj='EPSG:3857',
    geodesic=False
)


# Part 2: Defining Olga Strait geometry and filtering data

In [ ]:
# Define Olga Strait geometry (replace with actual coordinates)
olgastretet_geom = ee.Geometry.Polygon(
    [[[0, 80], [30, 80], [30, 78], [0, 78], [0, 80]]]
)

In [ ]:
for pol in polarizations:
    olgastretet_s1_dict[pol] = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(olgastretet_geom)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select(pol)
    .map(mask_edge)
    )

In [ ]:
# Sentinel-1 collection for Svalbard
svalbard_s1_dict = {}
polarizations = ['HH', 'VV', 'HV', 'VH']

for pol in polarizations:
    svalbard_s1_dict[pol] = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(svalbard_geom)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select(pol)
    .map(mask_edge)
    )


In [ ]:
svalbard_s1_pol = [
svalbard_s1HH := svalbard_s1_dict['HH'],
svalbard_s1VV := svalbard_s1_dict['VV'],
svalbard_s1HV := svalbard_s1_dict['HV'],
svalbard_s1VH := svalbard_s1_dict['VH']]

In [ ]:
# time ranges
spring = ee.Filter.date('2025-03-01', '2025-04-20')
late_spring = ee.Filter.date('2025-04-21', '2025-06-10')
summer = ee.Filter.date('2025-06-11', '2025-08-31')

seasons = [spring, late_spring, summer]

In [ ]:
Map = geemap.Map()
Map.centerObject(olgastretet_geom, zoom=6)
Map.addLayer(svalbard_s1HH, {'min': -25, 'max': 5}, 'Sentinel-1') # display a certain range of dB
Map

In [ ]:
# building the Sentinel-1 polarizations-seasons dataframe
season_names = ['spring', 'late_spring', 'summer']

svalbard_s1 = pd.DataFrame(
    index=polarizations,
    columns=season_names,
)

for pol in polarizations:
    for i, season_filter in enumerate(seasons):

        filtered_collection = svalbard_s1_dict[pol].filter(season_filter)

        svalbard_s1.loc[pol, season_names[i]] = filtered_collection

display(svalbard_s1)

In [ ]:
svalbard_s1_spring = [
svalbard_s1HH_spring := svalbard_s1.loc['HH', 'spring'],
svalbard_s1VV_spring := svalbard_s1.loc['VV', 'spring'],
svalbard_s1HV_spring := svalbard_s1.loc['HV', 'spring'],
svalbard_s1VH_spring := svalbard_s1.loc['VH', 'spring']]

svalbard_s1_latespring = [
svalbard_s1HH_late_spring := svalbard_s1.loc['HH', 'late_spring'],
svalbard_s1VV_late_spring := svalbard_s1.loc['VV', 'late_spring'],
svalbard_s1HV_late_spring := svalbard_s1.loc['HV', 'late_spring'],
svalbard_s1VH_late_spring := svalbard_s1.loc['VH', 'late_spring']]

svalbard_s1_summer = [
svalbard_s1HH_summer := svalbard_s1.loc['HH', 'summer'],
svalbard_s1VV_summer := svalbard_s1.loc['VV', 'summer'],
svalbard_s1HV_summer := svalbard_s1.loc['HV', 'summer'],
svalbard_s1VH_summer := svalbard_s1.loc['VH', 'summer']]

In [ ]:
"GEMINI HELPED"

def dates_with_data(collection, geometry):
    """ Finds the dates in which layers of a collection exist in a given geometry.

        Parameters:
        :: collection = layers collection
        :: geometry   = area of interest

        Returns:
        :: dates = list of dates with data (as 'yyyy-MM-ddTHH:mm:ss' strings)
    """

    # filter the collection by geometry
    filtered_collection = collection.filterBounds(geometry)

    # get a list of all image dates in the filtered collection
    # system:time_start is an attribute of every image (= starting time of aquisition)
    image_dates = ee.List(filtered_collection.aggregate_array('system:time_start'))

    # Check if there are any images
    if image_dates.size().getInfo() > 0:
        # Convert timestamps to dates and format them as strings
        dates = image_dates.map(lambda time: ee.Date(time).format('yyyy-MM-dd HH:mm:ss'))
        return dates.sort().getInfo()
    else:
        return None

# Find the dates with data for HH polarization in the spring collection
# Define the sea ice geometry using the provided diagonal vertices
# Coordinates are in EPSG:3857
sea_ice_geom = ee.Geometry.Polygon(
    [[[2501471, 14525773],
      [2572610, 14525773],
      [2572610, 14566293],
      [2501471, 14566293],
      [2501471, 14525773]]],
    proj='EPSG:3857',  # Specify the CRS here
    geodesic=False # Set to False for projected coordinates
)

dates_spring_HH = dates_with_data(svalbard_s1HH_spring, sea_ice_geom)

# Print the dates found
if dates_spring_HH:
    print(f"Dates with HH data in spring: {dates_spring_HH}")
else:
    print("No HH data found in spring for the specified geometry.")

In [ ]:
# Spring
SpringMap = geemap.Map()
SpringMap.centerObject(olgastretet_geom, zoom=6)

# Add mosaic layers for each polarization
for ic, pol in zip(svalbard_s1_spring, polarizations):
    ic_mosaic = ic.mosaic()
    SpringMap.addLayer(ic_mosaic, {'min': -25, 'max': 5}, f'{pol} Mosaic')

# Get the first date with HH data
ic_date_hh = svalbard_s1HH_spring.filterDate(ee.Date(1741187084000))
SpringMap.addLayer(ic_date_hh, {'min': -25, 'max': 5}, f'HH on {first_date_hh}')


SpringMap

In [ ]:
# Define the number of random points to generate initially (significantly more than needed)
initial_num_points = 15

# Generate random points within the bounds of the first Landsat 8 image
random_points = ee.FeatureCollection.randomPoints(sea_ice_geom, initial_num_points)

# Sample the image at the random points for all bands
sampled_data = first_l8_image.sampleRegions(
    collection=random_points,
    scale=30 # Landsat spatial resolution is 30 meters
)

# Convert the sampled data to a pandas DataFrame
sampled_df = geemap.ee_to_df(sampled_data)

# Filter out rows with missing values (NaNs) that might result from sampling no-data areas
sampled_df = sampled_df.dropna()

# Display the first few rows of the DataFrame
display(sampled_df)

# Create a map and add the image and sampled points
Map2 = geemap.Map()
vis_params = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 65535, 'gamma': 1.5} # Example visualization parameters
Map2.addLayer(first_l8_image, vis_params, 'First Landsat 8 Image')
Map2.addLayer(random_points, {'color': '00FF00'}, 'Sampled Points')
Map2.centerObject(first_l8_image.geometry(), 9) # Center map on the image
Map2